In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [28]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from sqlalchemy.ext.declarative import declarative_base

In [ ]:
#? what is diference from sqlite vs csv file | we can't read anything in sqlite, sqlite is a database file
#? create engine and connect is to connect the sqlite database to any apps using Python languages)
# step 1: data_path -> read that path by pd.read_sql() or engine.execute("select * from hwaii",connect) 
# step 2: create engine -> engine = create_engine(f"sqlite:///{data_path}") | engine = create_engine("sqlite://hawaii.sqlite")
# step 3: connect engine by connect = engine.connect() | if we wanna use Pandas for next steps, the object can be paste to Pandas
# Base = before going to define the Class |table 
# when to use DB Browser? to read sqlite file, to browse the sqlite file or even create table. 
# use session.query() is to read the file


In [4]:
# create engine to hawaii.sqlite

database_path = "Resources/hawaii.sqlite"
engine = create_engine(f"sqlite:///{database_path}")
# engine = create_engine("sqlite:///hawaii.sqlite")
# Base = declarative_base()

In [9]:
# reflect an existing database into a new model: sqlite is a database containing 2 tables that we connect then reflect them now
# ref. day 2, student reflection activity 
connect = engine.connect()
data_measurement = engine.execute("SELECT * FROM measurement")
data_station = engine.execute("SELECT * FROM station")

In [15]:
# View all of the classes that automap found : reflect 2 table available in DB Browser (measurement | station)
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['measurement', 'station']

In [14]:
# Save references to each table : class | bcoz the 2 tables are already available -> 
# no need to create Class __table__ : simpler way here is to use Base.classes.table_name
measurement = Base.classes.measurement
station = Base.classes.station

/var/folders/yn/xk1nllmx5_j6zx9bwj6p24kw0000gn/T/ipykernel_14934/581077833.py:2: SAWarning: This declarative base already contains a class with the same class name and module name as sqlalchemy.ext.automap.measurement, and will be replaced in the string-lookup table.
  Base.prepare(engine, reflect=True)
/var/folders/yn/xk1nllmx5_j6zx9bwj6p24kw0000gn/T/ipykernel_14934/581077833.py:2: SAWarning: This declarative base already contains a class with the same class name and module name as sqlalchemy.ext.automap.station, and will be replaced in the string-lookup table.
  Base.prepare(engine, reflect=True)


In [17]:
# Create our session (link) from Python to the DB : refer day 2 , 1 or 2 activity 
session = Session(engine)


# Exploratory Precipitation Analysis

In [18]:
# Find the most recent date in the data set.
session.query(measurement.date).order_by(measurement.date.desc()).first()

('2017-08-23',)

In [22]:
# Earliest Date
session.query(measurement.date).order_by(measurement.date).first()

('2010-01-01',)

In [20]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
session.query(measurement.date).\
    filter(measurement.date < '2017-08-23').\
    order_by(measurement.date).all()

[]

In [23]:
import datetime as dt

In [25]:
# Calculate the date one year from the last date in data set.
oneyear_date = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print("Date since one year from the last day is: ", oneyear_date)

Date since one year from the last day is:  2016-08-23


In [29]:
# Get a list of column names and types for reference first
inspector = inspect(engine)
inspector.get_table_names()
columns = inspector.get_columns('measurement')
for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [30]:
# same task with station table
inspector = inspect(engine)
inspector.get_table_names()
columns1 = inspector.get_columns('station')
for c in columns1:
    print(c['name'], c["type"])

id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [31]:
# Perform a query to retrieve the data and precipitation scores
# JOINs
session.query(measurement.station, station.station).limit(10).all()

/var/folders/yn/xk1nllmx5_j6zx9bwj6p24kw0000gn/T/ipykernel_14934/2826092912.py:2: SAWarning: SELECT statement has a cartesian product between FROM element(s) "station" and FROM element "measurement".  Apply join condition(s) between each element to resolve.
  session.query(measurement.station, station.station).limit(10).all()


[('USC00519397', 'USC00519397'),
 ('USC00519397', 'USC00513117'),
 ('USC00519397', 'USC00514830'),
 ('USC00519397', 'USC00517948'),
 ('USC00519397', 'USC00518838'),
 ('USC00519397', 'USC00519523'),
 ('USC00519397', 'USC00519281'),
 ('USC00519397', 'USC00511918'),
 ('USC00519397', 'USC00516128'),
 ('USC00519397', 'USC00519397')]

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column


In [ ]:
# Sort the dataframe by date


In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()